# Model 1 - Minimize Transaction Size

## Problem Modelling

* **Input parameters**

| ________________Variable______________________  | Meaning                       |
|-----------------------------------|-------------------------------|
| $U = \{u_1, \ldots, u_n\}$        | set of UTXOs                  |
| $O = \{o_1, \ldots, o_m\}$        | set of outputs                |
| $V^u = \{v_1^u, \ldots, v_n^u \}$ | set of value of UTXOs         |
| $V^o = \{v_1^o, \ldots, v_m^o \}$ | set of value of outputs       |
| $S^u = \{s_1^u, \ldots, s_n^u \}$ | set of size of UTXOs          |
| $S^o = \{s_1^o, \ldots, s_m^o \}$ | set of size of outputs        |
| $M_s$                             | maximum size of a transaction |
| $\alpha$                          | fee rate                      |
| $T$                               | dust threshold                |
| $\epsilon$                        | minimum change output         |

* **Decision variables**:
$$
x_i =
\begin{cases}
1, \text{if UTXO $u_i$ is chosen} \\
0, \text{otherwise}
\end{cases}
$$

* **Immediate variables**:
  * $y$: transaction size
  * $z_v$: change output
  * $z_s$: change value

* The relationship between change output value $z_v$ and its size $z_s$:
$$
z_s =
\begin{cases}
0, 0 \leq z_v \leq \epsilon \\
\beta, z_v > \epsilon
\end{cases}
$$

* **Constraint 1**: A transaction size may not exceed maximum block data size.
$$
y = \sum\limits_{i|u_i \in U} (s_i^u * x_i) + \sum\limits_{j|o_i \in O} s_j^o + z_s \leq M_s
$$

* **Constraint 2**: A transaction must have sufficient value for consuming.
$$
\sum\limits_{i|u_i \in U} (v_i^u * x_i) = \sum\limits_{j|o_i \in O} v_j^o + \alpha y + z_v
$$

* **Constraint 3**: All the transaction outputs must be higher than the dust threshold to certain that this transaction is relayed to the network and confirmed.
$$
\sum\limits_{j|o_i \in O} v_j^o \geq T
$$


* **Objective function**: Minimize transaction size
$$
min(y)
$$


## Reformulating

* Define $M_c$ as the maximum value of change.
$$
M_{c} = \sum\limits_{i|u_i \in U} (v_i^u * x_i) - \sum\limits_{j|o_i \in O} v_j^o
$$

* Define a binary variable $t$:
$$
    t =
    \begin{cases}
        0, \text{if $z_v - \epsilon \leq 0$, or if $z_s = 0$}\\
        1, \text{if $z_v - \epsilon > 0$, or if $z_s = \beta$}
    \end{cases}
$$

* The relationship between $t$ and $z_v$ can be rewritten as a linear inequality:
$$
    z_v - \epsilon \leq M_c t \\
\Rightarrow
    - M_c t + z_v \leq \epsilon
$$

* Substitute $t$ into $y$:
$$
y = \sum\limits_{i|u_i \in U} (s_i^u * x_i) + \sum\limits_{j|o_i \in O} s_j^o + t \times \beta \leq M_s
$$

* Substitute $t$ into constraint 1:
$$
    \sum\limits_{i|u_i \in U} (v_i^u * x_i) 
    =
    \sum\limits_{j|o_i \in O} v_j^o + \alpha y + z_v
$$
$$
\Rightarrow
    \sum\limits_{i|u_i \in U} (v_i^u * x_i)
    =
    \sum\limits_{j|o_i \in O} v_j^o + \alpha \bigg[\sum\limits_{i|u_i \in U} (s_i^u * x_i) + \sum\limits_{j|o_i \in O} s_j^o + t \times \beta\bigg] + z_v
$$
$$
\Rightarrow
    \sum\limits_{i|u_i \in U} \bigg[\bigg(v_i^u - \alpha s_i^u\bigg) x_i \bigg] - \alpha \beta t - z_v
    =
    \sum\limits_{j|o_i \in O} v_j^o + \alpha \sum\limits_{j|o_i \in O} s_j^o \\
$$

## Solving with MOSEK
After reformulating, the model can be solved by linear optimization using MOSEK:

Minimize the objective function:
$$
y = \sum\limits_{i|u_i \in U} (s_i^u * x_i) + \sum\limits_{j|o_i \in O} s_j^o + t \times \beta
$$

subject to the linear constraints:
$$
\sum\limits_{i|u_i \in U} (s_i^u * x_i) + \sum\limits_{j|o_i \in O} s_j^o + t \times \beta \leq M_s
$$

$$
    \sum\limits_{i|u_i \in U} \bigg[\bigg(v_i^u - \alpha s_i^u\bigg) x_i \bigg] - \alpha \beta t - z_v
    =
    \sum\limits_{j|o_i \in O} v_j^o + \alpha \sum\limits_{j|o_i \in O} s_j^o
$$

$$
-M_c t + z_v \leq \epsilon
$$

and the bounds:

$$
    x_i \in \{0, 1\} \forall u_i \in U \\
    t_i \in \{0, 1\} \\
    0 \leq z_v \leq M_c
$$

In [28]:
import csv
import glob
import time
import sys

import mosek

import meta
import preprocess
from transaction import Transaction


In [29]:
tx_indices = meta.get_all_tx_indices()

In [30]:
tx = Transaction(tx_indices[0])
tx.load_params()
tx.load_inputs()
tx.load_outputs()

In [31]:
n_utxo = len(tx.utxo_set)
n_out  = len(tx.outputs)
U      = tx.utxo_set
O      = tx.outputs
Vu     = [int(U[j]['value']) for j in range(n_utxo)]
Vo     = [int(O[j]['value']) for j in range(n_out)]
Su     = [int(U[j]['size'])  for j in range(n_utxo)]
So     = [int(O[j]['size'])  for j in range(n_out)]

Ms     = int(tx.params['max_size'])
alpha  = float(tx.params['fee_rate'])
T      = int(tx.params['dust_threshold'])
eps    = int(tx.params['min_change_output'])
beta   = int(tx.params['change_output_size'])

print("Vu    = {0}".format(Vu))
print("Vo    = {0}".format(Vo))
print("Su    = {0}".format(Su))
print("So    = {0}".format(So))
print("Ms    = {0}".format(Ms))
print("alpha = {0}".format(alpha))
print("T     = {0}".format(T))
print("eps   = {0}".format(eps))
print("beta  = {0}".format(beta))

Vu    = [12704661, 971963, 1390321, 12790142, 6336294, 12805967, 2078617, 54277780, 7503421, 1802053, 12736951, 1749705, 38399352, 12778105, 2622991, 78851078, 9227777, 12467603, 1192428, 11578033, 3722314, 1237213, 1532714, 2229172, 1964909, 1440408, 1534468, 1336713, 1210283, 8260429, 9327588, 8460234, 10204192, 1251571, 1341721, 1641761, 1243865, 1304350, 1294104, 1225931, 1304321, 1229145, 1219208, 1296721, 1225604, 1311093, 1688423, 1296175, 1218407, 2034878, 1303427, 1713788, 1695048, 1520310, 1568162, 1469958, 1209649, 1416007, 1696680, 1654032, 1316053, 1045409, 1350278, 2242165, 3131190, 7556331, 3422634, 7493461, 6588404, 3150612, 1728139, 1547942, 2058045, 5677311, 1273962, 6605932, 6179948, 3697422, 3259590, 3216818, 6178689, 3254800, 3334892, 5145055, 2539945, 3768050, 2830591, 6642083, 3537790, 2791020, 2034580, 2227194, 2619658, 2233145, 2515490, 1795803, 2399638, 2759259, 1968086]
Vo    = [12266303]
Su    = [148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 148, 14

In [32]:
# Sum of input values
sum_Vu = sum(Vu[j] for j in range(n_utxo))
# Sum of output values
sum_Vo = sum(Vo[j] for j in range(n_out))
# Sum of output size
sum_So = sum(So[j] for j in range(n_out))
# Minimum utxo
min_Vu = min(Vu)

print("sum_Vu = {0}".format(sum_Vu))
print("sum_Vo = {0}".format(sum_Vo))
print("sum_So = {0}".format(sum_So))

sum_Vu = 518217632
sum_Vo = 12266303
sum_So = 34


In [33]:
# Maximum change value = sum_Vu - sum_Vo
Mc = sum_Vu - sum_Vo

print(Mc)

505951329


In [34]:
%%javascript
MathJax.Hub.Config({
    TeX: { equationNumbers: { autoNumber: "AMS" } }
});

<IPython.core.display.Javascript object>

In [35]:
print(Mc - sum_So)
print(sum_So)

505951295
34


In [36]:
assert T <= sum_Vo
print(T)
print(sum_Vo)

7359
12266303


In [37]:
# Since the actual value of Infinity is ignored, we define it solely
# for symbolic purposes:
inf = 0.0

def streamprinter(text):
    sys.stdout.write(text)
    sys.stdout.flush()

def solve():
    # Make mosek environment
    with mosek.Env() as env:
        # Create a task object
        with env.Task(0, 0) as task:
            # Attach a log stream printer to the task
            task.set_Stream(mosek.streamtype.log, streamprinter)

            # Bound keys for constraints
            bkc = [
                mosek.boundkey.ra,
                mosek.boundkey.fx,
                mosek.boundkey.up
            ]

            # Bound values for constraints
            blc = [0.0,         sum_Vo + alpha * sum_So, -inf]
            buc = [Ms - sum_So, sum_Vo + alpha * sum_So, eps]


            # Bound keys for variables
            bkx = [mosek.boundkey.ra for j in range(n_utxo)] # l <= x <= u
            bkx.append(mosek.boundkey.ra)
            bkx.append(mosek.boundkey.ra)


            # Bound values for variables
            blx = [0 for j in range(n_utxo)] # x_i
            blx.append(0)                    # t
            blx.append(0)                    # z_v
            bux = [1 for j in range(n_utxo)]
            bux.append(1)
            bux.append(min_Vu)

            # Objective coefficients
            c = [Su[j] for j in range(n_utxo)]
            c.append(beta)
            c.append(0.0)

            # Below is the sparse representation of the A
            # matrix stored by column.
            asub = [
                [0, 1] for j in range(n_utxo)
            ]
            asub.append([0, 1, 2])
            asub.append([1, 2])

            aval = [
                [Su[j], Vu[j] - alpha * Su[j]] for j in range(n_utxo)
            ]
            aval.append([beta, -alpha * beta, -min_Vu])
            aval.append([-1, 1])


            # numcon and numvar
            numcon = len(bkc)
            numvar = len(bkx)

            assert numvar == n_utxo + 2
            assert numcon == 3

            # Append 'numcon' empty constraints.
            # The constraints will initially have no bounds.
            task.appendcons(numcon)
            # Append 'numvar' variables.
            # The variables will initially be fixed at zero (x=0).
            task.appendvars(numvar)

            for j in range(numvar):
                # Set the linear term c_j in the objective.
                task.putcj(j, c[j])

                # Set the bounds on variable j
                # blx[j] <= x_j <= bux[j]
                task.putvarbound(j, bkx[j], blx[j], bux[j])

                # Input column j of A
                task.putacol(
                    j,       # Variable (column) index.
                    asub[j], # Row index of non-zeros in column j.
                    aval[j]  # Non-zero Values of column j.
                )


            task.putconboundlist(range(numcon), bkc, blc, buc)

            # Input the objective sense (minimize/maximize)
            task.putobjsense(mosek.objsense.minimize)

            # Define variables to be integers
            task.putvartypelist(
                [j for j in range(n_utxo + 1)],
                [mosek.variabletype.type_int for j in range(n_utxo + 1)]
            )

            # Set max solution time
            task.putdouparam(mosek.dparam.mio_max_time, 60.0);

            # Solve the problem
            task.optimize()

            # Print a summary containing information
            # about the solution for debugging purposes
            task.solutionsummary(mosek.streamtype.msg)
            prosta = task.getprosta(mosek.soltype.itg)
            solsta = task.getsolsta(mosek.soltype.itg)

            # Output a solution
            xx = [0.] * numvar
            task.getxx(mosek.soltype.itg, xx)

            if solsta in [mosek.solsta.integer_optimal]:
                print("Optimal solution: %s" % xx)
                return xx
            elif solsta == mosek.solsta.prim_feas:
                print("Feasible solution: %s" % xx)
            elif mosek.solsta.unknown:
                if prosta == mosek.prosta.prim_infeas_or_unbounded:
                    print("Problem status Infeasible or unbounded.\n")
                elif prosta == mosek.prosta.prim_infeas:
                    print("Problem status Infeasible.\n")
                elif prosta == mosek.prosta.unkown:
                    print("Problem status unkown.\n")
                else:
                    print("Other problem status.\n")
            else:
                print("Other solution status")


# call the solve function
try:
    result = solve()

except mosek.Error as e:
    print("ERROR: %s" % str(e.errno))
    if e.msg is not None:
        print("\t%s" % e.msg)
        sys.exit(1)
except:
    import traceback
    traceback.print_exc()
    sys.exit(1)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 3               
  Cones                  : 0               
  Scalar variables       : 101             
  Matrix variables       : 0               
  Integer variables      : 100             

Optimizer started.
Mixed integer optimizer started.
Threads used: 4
Presolve started.
Presolve terminated. Time = 0.00
Presolved problem: 98 variables, 2 constraints, 100 non-zeros
Presolved problem: 0 general integer, 97 binary, 1 continuous
Clique table size: 1
BRANCHES RELAXS   ACT_NDS  DEPTH    BEST_INT_OBJ         BEST_RELAX_OBJ       REL_GAP(%)  TIME  
0        1        0        0        NA                   1.4200000000e+02     NA          0.0   
0        1        0        0        1.8200000000e+02     1.4200000000e+02     21.98       0.0   
Cut generation started.
0        2        0        0        1.820000

In [38]:
print(eps)

7359


In [39]:
# Evaluate result
selected = [round(result[j]) for j in range(n_utxo)]
n_selected = sum(selected[j] for j in range(n_utxo))
change_val = round(result[-1])
tx_size    = n_selected * 148 + n_out * 34 + (34 if change_val > 0 else 0)
tx_fee     = round(alpha * tx_size)

print("Result:")
print("n_selected = {0}".format(n_selected))
print("change_val = {0}".format(change_val))
print("tx_size    = {0}".format(tx_size))
print("tx_fee     = {0}".format(tx_fee))

Result:
n_selected = 1
change_val = 536753
tx_size    = 216
tx_fee     = 2911
